In [1]:
import assets.helper as b3
import assets.functions as run


In [2]:
# def construct_graphs(df):
#     # Step 1: Initialize the main dictionary
#     graphs_manual = {}


#     # Step 2: Add the first section key to the main dictionary
#     graphs_manual['01'] = {}

#     # Step 3: Add the first line key to the first section
#     graphs_manual['01'][0] = {}

#     # Step 4: Add the first plot key-value to the first line

#     line = '01 - Ativo Total'
#     graphs_manual['01'][0][0] = {
#         'info': {
#             'title': f'{line}',
#             'header': f'{line}',
#             'footer': f'{line}',
#             'description': f'Linha {line} do balanço estruturado. Graph generator'
#         },
#         'data': {
#             'axis': ['Reais (R$)', 'Porcentagem (%)'],
#             'left_1': [line],
#             'right': [run.cagr(line, df)], 
#             # 'right_2': [line],
#         },
#         'options': {
#             'left_1': {'shape': 'line', 'mode': 'standalone', 'normalization': False, 'mma': [3, 2]},
#             'right': {'shape': 'line', 'mode': 'standalone', 'normalization': False, 'outliers': False, 'range': 'flexible'}, 
#             # 'right_2': {'shape': 'area', 'mode': 'cumulative', 'normalization': True, },
#         }, 
#     }

#     sub_lines = [column for column in df.columns if column.startswith(line.split(' - ')[0] + '.') and column.count('.') == line.count('.') + 1]
#     graphs_manual['01'][0][1] = {
#         'info': {
#             'title': f'{line} - Composição Relativa',
#             'header': f'{line} header relativo',
#             'footer': f'{line} footer relativo',
#             'description': f'{line} description relativa',
#         }, 
#         'data': {
#             'axis': ['Porcentagem (%)', 'Porcentagem (%)'],
#             'left': sub_lines,
#         },
#         'options': {
#             'left': {'shape': 'line', 'mode': 'standalone', 'normalization': True,},
#         }
#     }

#     # # -----

#     # # Step 2: Add the first section key to the main dictionary
#     # graphs_manual['02'] = {}

#     # # Step 3: Add the first line key to the first section
#     # graphs_manual['02'][0] = {}

#     # # Step 4: Add the first plot key-value to the first line

#     # line = '02 - Passivo Total'
#     # graphs_manual['02'][0][0] = {
#     #     'info': {
#     #         'title': f'{line}',
#     #         'header': f'{line}',
#     #         'footer': f'{line}',
#     #         'description': f'Linha {line} do balanço estruturado. Graph generator'
#     #     },
#     #     'data': {
#     #         'axis': ['Reais (R$)', 'Porcentagem (%)'],
#     #         'left': [line],
#     #         'right': [run.cagr(line, df)]
#     #     },
#     #     'options': {
#     #         'left': {'shape': 'area', 'mode': 'standalone', 'normalization': False, 'mma': [3, 2]},
#     #         'right': {'shape': 'line', 'mode': 'standalone', 'normalization': False, 'outliers': False, 'range': 'flexible'}, 
#     #     }, 
#     # }

#     # sub_lines = [column for column in df.columns if column.startswith(line.split(' - ')[0] + '.') and column.count('.') == line.count('.') + 1]

#     # graphs_manual['02'][0][1] = {
#     #     'info': {
#     #         'title': f'{line} - Composição Relativa',
#     #         'header': f'{line} header relativo',
#     #         'footer': f'{line} footer relativo',
#     #         'description': f'{line} description relativa',
#     #     }, 
#     #     'data': {
#     #         'axis': ['Reais (R$)', 'Porcentagem (%)'],
#     #         'left': sub_lines,
#     #         # 'right': ['PLACEHOLDER_for_run.cagr_function_result'],  # Placeholder for run.cagr function
#     #     },
#     #     'options': {
#     #         'left': {'shape': 'line', 'mode': 'standalone', 'normalization': False,},
#     #         # 'right': {'shape': 'line', 'mode': 'standalone', 'normalization': False, 'outliers': False, 'range': 'flexible'}, 
#     #     }
#     # }

#     # # -----

#     # # Step 2: Add the first section key to the main dictionary
#     # graphs_manual['03'] = {}

#     # # Step 3: Add the first line key to the first section
#     # graphs_manual['03'][0] = {}

#     # # Step 4: Add the first plot key-value to the first line

#     # line = '03.11 - Lucro Líquido'
#     # graphs_manual['03'][0][0] = {
#     #     'info': {
#     #         'title': f'{line}',
#     #         'header': f'{line}',
#     #         'footer': f'{line}',
#     #         'description': f'Linha {line} do balanço estruturado. Graph generator'
#     #     },
#     #     'data': {
#     #         'axis': ['Reais (R$)', 'Porcentagem (%)'],
#     #         'left': [line],
#     #         'right': [run.cagr(line, df)]
#     #     },
#     #     'options': {
#     #         'left': {'shape': 'area', 'mode': 'standalone', 'normalization': False, 'mma': [3, 2]},
#     #         'right': {'shape': 'line', 'mode': 'standalone', 'normalization': False, 'outliers': False, 'range': 'flexible'}, 
#     #     }, 
#     # }

#     # sub_lines = [column for column in df.columns if column.startswith(line.split(' - ')[0] + '.') and column.count('.') == line.count('.') + 1]

#     # graphs_manual['03'][0][1] = {
#     #     'info': {
#     #         'title': f'{line} - Composição Relativa',
#     #         'header': f'{line} header relativo',
#     #         'footer': f'{line} footer relativo',
#     #         'description': f'{line} description relativa',
#     #     }, 
#     #     'data': {
#     #         'axis': ['Reais (R$)', 'Porcentagem (%)'],
#     #         'left': sub_lines,
#     #         # 'right': ['PLACEHOLDER_for_run.cagr_function_result'],  # Placeholder for run.cagr function
#     #     },
#     #     'options': {
#     #         'left': {'shape': 'line', 'mode': 'standalone', 'normalization': False,},
#     #         # 'right': {'shape': 'line', 'mode': 'standalone', 'normalization': False, 'outliers': False, 'range': 'flexible'}, 
#     #     }
#     # }
#     return graphs_manual



In [98]:
df = run.load_pkl('df')
df.head()
df.columns.to_list()

['PREGAO',
 'Open',
 'High',
 'Low',
 'Close',
 'Adj Close',
 'Volume',
 'TICKER',
 'DT_REFER',
 'FILENAME',
 'DEMONSTRATIVO',
 'BALANCE_SHEET',
 'ANO',
 'AGRUPAMENTO',
 'CNPJ_CIA',
 'VERSAO',
 'DENOM_CIA',
 'CD_CVM',
 'GRUPO_DFP',
 'MOEDA',
 'ESCALA_MOEDA',
 'DT_FIM_EXERC',
 'ST_CONTA_FIXA',
 'DT_INI_EXERC',
 'COLUNA_DF',
 'COMPANHIA',
 'TICK',
 'LISTAGEM',
 'CVM',
 'TICKERS',
 'ISIN',
 'CNPJ',
 'ATIVIDADE',
 'SETOR',
 'SUBSETOR',
 'SEGMENTO',
 'SITE',
 'ESCRITURADOR',
 'CD_CONTA_original',
 'DS_CONTA_original',
 '00.01.01 - Ações ON',
 '00.01.02 - Ações ON em Tesouraria',
 '00.02.01 - Ações PN',
 '00.02.02 - Ações PN em Tesouraria',
 '01 - Ativo Total',
 '01.01 - Ativo Circulante de Curto Prazo',
 '01.01.01 - Caixa e Disponibilidades de Caixa',
 '01.01.02 - Aplicações Financeiras',
 '01.01.03 - Contas a Receber',
 '01.01.04 - Estoques',
 '01.01.05 - Ativos Biológicos',
 '01.01.06 - Tributos',
 '01.01.07 - Despesas',
 '01.01.09 - Outros Ativos Circulantes',
 '01.02 - Ativo Não Circula

In [79]:
def generate_graph_items(lines_info, df):
    """
    Generate graph items based on the provided lines information.

    Parameters:
    - df: DataFrame containing the data.
    - lines_info: A list of dictionaries containing info about each line. Each dictionary should have:
        - 'line': The main line item.
        - 'title': Title for the graph.
        - 'header': Header for the graph.
        - 'footer': Footer for the graph.
        - 'description': Description for the graph.

    Returns:
    - A dictionary with the generated graph items.
    """
    # Extract unique first two digits from the line string from each entry in lines_info."""
    groups = set(entry['line'][:2] for entry in lines_info)
    groups = sorted(groups)
    
    graphs_manual = {}
    for g, group in enumerate(groups):
        # print(f'g {g}, group {group}')
        filtered_lines = [entry for entry in lines_info if entry['line'].startswith(group)]
        graphs_manual[group] = {}
        for l, line in enumerate(filtered_lines):
            print(f'g {group} l {l}, line {line["line"]}')
            graphs_manual[group][l] = {}
            graphs_manual[group][l][0] = {
                'info': {
                    'title': line["title"], 
                    'header': line["header"], 
                    'description': line["description"], 
                    'footer': line["footer"], 
                }, 
                'data': {
                    'axis': ['Reais (RS)', 'Porcentagem (%)'], 
                    'left': [line["line"]], 
                    'right': [run.cagr(line, df)], 
                }, 
                'options': {
                    'left': {'shape': 'line', 'mode': 'standalone', 'normalization': False, 'mma': [3, 2], },
                    'right': {'shape': 'line', 'mode': 'standalone', 'normalization': False, 'outliers': False, }, 
                }, 
            }
            sublines = [column for column in df.columns if column.startswith(line["line"].split(' - ')[0] + '.') and column.count('.') == line["line"].count('.') + 1]
            if sublines: 
                graphs_manual[group][l][1] = {
                    'info': {
                        'title': line["title"], 
                        'header': line["header"], 
                        'description': line["description"], 
                        'footer': line["footer"], 
                    }, 
                    'data': {
                        'axis': ['Porcentagem (%)', 'Porcentagem (%)'],
                        'left': sublines,
                    },
                    'options': {
                        'left': {'shape': 'line', 'mode': 'standalone', 'normalization': True,},
                    }
                }


    return graphs_manual



In [77]:
lines_info = [
    {
        'line': '01 - Ativo Total',
        'title': 'Some Title 1',
        'header': 'Some Header 1',
        'footer': 'Some Footer 1',
        'description': 'Some Description 1', 
    },
    {
        'line': '02 - Passivo Total',
        'title': 'Some Title 2',
        'header': 'Some Header 2',
        'footer': 'Some Footer 2',
        'description': 'Some Description 2', 
    },
    {
        'line': '01.01 - subitem do Ativo Total',
        'title': 'Some Title 1.1',
        'header': 'Some Header 1.1',
        'footer': 'Some Footer 1.1',
        'description': 'Some Description 1.1', 
    },
    {
        'line': '01.01.01.01 - subitem do Ativo Total',
        'title': 'Some Title 1.1.1.1',
        'header': 'Some Header 1.1.1.1',
        'footer': 'Some Footer 1.1.1.1',
        'description': 'Some Description 1.1.1.1', 
    },
]



In [94]:
line = lines_info[0]
line['line']

'01 - Ativo Total'

In [97]:
for line in lines_info:
    print(line["line"])
    try:
        data_series = run.cagr(line["line"], df)
        print(data_series[-1])
    except:
        pass

01 - Ativo Total
20.08
02 - Passivo Total
20.08
01.01 - subitem do Ativo Total
01.01.01.01 - subitem do Ativo Total


In [83]:
graphs_manual = generate_graph_items(lines_info, df)


g 01 l 0, line 01 - Ativo Total
g 01 l 1, line 01.01 - subitem do Ativo Total
g 01 l 2, line 01.01.01.01 - subitem do Ativo Total
g 02 l 0, line 02 - Passivo Total
